Trying with Pyviz

Link to the Tutorial: [PyVis](https://pyvis.readthedocs.io/en/latest/tutorial.html)

In [50]:
!pip install pyvis==0.3.1
from pyvis.network import Network
import pandas as pd
from pandas import DataFrame
import textwrap

def extract_numbers(input_string):
    numbers_only = ''.join(char for char in input_string if char.isdigit())
    result_integer = int(numbers_only)
    return result_integer

def wrap_text(text, width=50):
    wrapper = textwrap.TextWrapper(width=width, break_long_words=False)
    return '\n'.join(wrapper.wrap(text))

In [51]:
MainSheet = pd.read_excel("dt.xlsx", sheet_name='main', skiprows=0)
FeatureDescriptionSheet = pd.read_excel("dt.xlsx", sheet_name='FeatureDescription', skiprows=0)
ClassDescriptionSheet = pd.read_excel("dt.xlsx", sheet_name='ClassDescription', skiprows=0)
EntityDescriptionSheet = pd.read_excel("dt.xlsx", sheet_name='EntityDescription', skiprows=0)

MainSheet['ReACT_title'] = MainSheet['ReACT_title'].apply(wrap_text)


ReAct_Dict = dict(zip(MainSheet['ReACT_number'], MainSheet['ReACT_title']))
FeatureDescription_Dict = dict(zip(FeatureDescriptionSheet['FeatureName'], FeatureDescriptionSheet['DescriptionOfTheFeature']))
ClassDescription_Dict = dict(zip(ClassDescriptionSheet['Class'], ClassDescriptionSheet['ClassDescription']))
EntityDescription_Dict = dict(zip(EntityDescriptionSheet['Entity'], EntityDescriptionSheet['EntityDescription']))
MainSheet.head()

,ReACT_number,ReACT_title,Class,Entity_responsible,Features,Quatative/Qualatitative,References
0,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",Code,"Project Maintainers, Project Leaders","num_act_dev, top_e_fract, top_c_fract",NaN,No. of reference: 3\n
1,ReACT-1,Maintaining a set of active/friendly developers,"Individual, Group","Project Maintainers, Project Leaders",num_act_dev,NaN,No. of reference: 2\n
2,ReACT-6,Maintaining Continuous Integration,Code,"Project Leaders, Core Developers",num_act_dev,NaN,No. of reference: 2\n
3,ReACT-11,Maintaining project small or simple,"Project, Ecosystem",Core Developers,"num_act_dev, top_c_fract",NaN,No. of reference: 2\n
4,ReACT-34,Keeping small number of core developers / (Hig...,Project,"Project Maintainers, Technical Leads, Communit...","num_act_dev, top_e_fract, top_c_fract",NaN,No. of reference: 2\n


In [52]:
ReActAndTitle = MainSheet[['ReACT_number', 'ReACT_title']]

#Plotting ReAct and Class - Network Diagram

In [53]:
ReActAndClass = MainSheet[['ReACT_number', 'Class']]

b = DataFrame(ReActAndClass.Class.str.split(',').tolist(), index=ReActAndClass.ReACT_number).stack()
b = b.reset_index()[[0, 'ReACT_number']]
b.columns = ['Class', 'ReACT_number'] # renaming var1
ReActAndClass = b

In [54]:
ReActTitleClass = pd.merge(ReActAndTitle, ReActAndClass, on='ReACT_number', how='outer')
ReActTitleClass['ClassDescription'] = None

ReActClassDescription = ReActTitleClass
ReActClassDescription
for i in range(0,len(ReActClassDescription)):
  # print(ReActClassDescription.Class[i].lstrip().rstrip())
  # print(i, ClassDescription_Dict[ReActClassDescription.Class[i].lstrip().rstrip()])
  ReActClassDescription.ClassDescription[i] = ClassDescription_Dict[ReActClassDescription.Class[i].lstrip().rstrip()]
ReAct_Title_Class_ClassDescription = ReActClassDescription


In [55]:
ReACT_number = ReAct_Title_Class_ClassDescription['ReACT_number'].values
ReACT_Title = ReAct_Title_Class_ClassDescription['ReACT_title'].values
ReAct_Title_Class_ClassDescription['ReACT_title'] = ReAct_Title_Class_ClassDescription['ReACT_title'].apply(wrap_text)

Class = ReAct_Title_Class_ClassDescription['Class'].values
ReAct_Title_Class_ClassDescription['ClassDescription'] =  ReAct_Title_Class_ClassDescription['ClassDescription'].apply(wrap_text)
ClassDescription = ReAct_Title_Class_ClassDescription['ClassDescription'].values

net = Network(directed =True)
for i in range(0,len(ReACT_number)):
  # print(actionable[i])
  net.add_node(ReACT_number[i], label=ReACT_number[i], title=ReACT_Title[i])
  net.add_node(Class[i], label=Class[i], title=ClassDescription[i], color = "#5DBB63")
  net.add_edge(ReACT_number[i], Class[i], color = "#016064")

net.show('interactive_graph_classes.html')

#Plotting ReAct and Entity - Network Diagram

In [56]:
ReActAndEntity = MainSheet[['ReACT_number', 'Entity_responsible']]

b = DataFrame(ReActAndEntity.Entity_responsible.str.split(',').tolist(), index=ReActAndEntity.ReACT_number).stack()
b = b.reset_index()[[0, 'ReACT_number']]
b.columns = ['Entity', 'ReACT_number'] # renaming var1
ReActAndEntity = b
ReActAndEntity.Entity

0       Project Maintainers
1           Project Leaders
2       Project Maintainers
3           Project Leaders
4           Project Leaders
               ...         
107         Project Leaders
108              Developers
109            Contributors
110      Community Managers
111     Project Maintainers
Name: Entity, Length: 112, dtype: object

###Unique values in the Entity Column
(You neeed description for each item in the EntityDescription Sheet)

In [57]:
unique_values = set(ReActAndEntity.Entity)

# Convert the unique values back to a list if needed
unique_values_list = list(unique_values)

print(unique_values_list)
for i in range(0,len(unique_values_list)):
  print(unique_values_list[i].lstrip().rstrip())

['Developers', 'Sponsors', 'Project Leaders', ' Developers', ' Owners', ' Project Maintainers', ' Technical Leads', ' Project Leaders', 'Project Managers', 'Architects', 'Community Moderators', ' Design Team', ' Collaboration Coordinators', 'Contributors', ' Peer Reviewers', 'Core Developers', 'Mentors', ' Quality Assurance Team', ' Documentation Team', '\n Contributors', ' Core Developers', ' Sponsors', 'Project Maintainers', ' Contributors', 'Community Managers', ' Community Managers']
Developers
Sponsors
Project Leaders
Developers
Owners
Project Maintainers
Technical Leads
Project Leaders
Project Managers
Architects
Community Moderators
Design Team
Collaboration Coordinators
Contributors
Peer Reviewers
Core Developers
Mentors
Quality Assurance Team
Documentation Team
Contributors
Core Developers
Sponsors
Project Maintainers
Contributors
Community Managers
Community Managers


In [58]:
ReActTitleEntity = pd.merge(ReActAndTitle, ReActAndEntity, on='ReACT_number', how='outer')
ReActTitleEntity['EntityDescription'] = None

ReActEntityDescription = ReActTitleEntity

for i in range(0,len(ReActEntityDescription)):
  ReActEntityDescription.EntityDescription[i] = EntityDescription_Dict[ReActEntityDescription.Entity[i].lstrip().rstrip()]
ReAct_Title_Entity_EntityDescription = ReActEntityDescription
ReAct_Title_Entity_EntityDescription

,ReACT_number,ReACT_title,Entity,EntityDescription
0,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",Project Maintainers,Project maintainers in open-source software de...
1,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",Project Leaders,Project leaders in open-source software develo...
2,ReACT-1,Maintaining a set of active/friendly developers,Project Maintainers,Project maintainers in open-source software de...
3,ReACT-1,Maintaining a set of active/friendly developers,Project Leaders,Project leaders in open-source software develo...
4,ReACT-6,Maintaining Continuous Integration,Project Leaders,Project leaders in open-source software develo...
...,...,...,...,...
107,ReACT-48,Plan to finish the project in a shorter amount...,Project Leaders,Project leaders in open-source software develo...
108,ReACT-48,Plan to finish the project in a shorter amount...,Developers,"In open-source software development, developer..."
109,ReACT-49,Incorporating inclusive pronouns in communicat...,Contributors,Contributors in open-source software developme...
110,ReACT-49,Incorporating inclusive pronouns in communicat...,Community Managers,Community managers in open source oversee posi...


In [59]:
df = ReAct_Title_Entity_EntityDescription
ReACT_number = df['ReACT_number'].values

ReACT_Title = df['ReACT_title'].values

Entity = df['Entity'].values
df['EntityDescription'] = df['EntityDescription'].apply(wrap_text)
EntityDescription = df['EntityDescription'].values


net = Network(directed =True)
for i in range(0,len(ReACT_number)):
  # print(actionable[i])
  net.add_node(ReACT_number[i], label=ReACT_number[i], title=ReACT_Title[i])
  net.add_node(Entity[i], label=Entity[i], title=EntityDescription[i], color = "#98BF64")
  net.add_edge(Entity[i], ReACT_number[i], color = "#32612D")

net.show('interactive_graph_Entity.html')

#Plotting ReAct and Features - Network Diagram

In [60]:
ReActAndFeature = MainSheet[['ReACT_number', 'Features']]

b = DataFrame(ReActAndFeature.Features.str.split(',').tolist(), index=ReActAndFeature.ReACT_number).stack()
b = b.reset_index()[[0, 'ReACT_number']]
b.columns = ['Features', 'ReACT_number'] # renaming var1
ReActAndFeature  = b
ReActAndFeature

,Features,ReACT_number
0,num_act_dev,ReACT-3
1,top_e_fract,ReACT-3
2,top_c_fract,ReACT-3
3,num_act_dev,ReACT-1
4,num_act_dev,ReACT-6
...,...,...
75,top_e_fract,ReACT-47
76,top_c_fract,ReACT-47
77,top_e_fract,ReACT-48
78,top_c_fract,ReACT-48


In [61]:
ReActTitleFeature = pd.merge(ReActAndTitle, ReActAndFeature, on='ReACT_number', how='outer')
ReActTitleFeature['FeatureDescription'] = None
ReActAndReference = MainSheet[['ReACT_number', 'References']]

ReActFeatureDescription = ReActTitleFeature


for i in range(0,len(ReActFeatureDescription)):
 ReActFeatureDescription.FeatureDescription[i] = FeatureDescription_Dict[ReActFeatureDescription.Features[i].lstrip().rstrip()]
ReAct_Title_Feature_FeatureDescription = ReActFeatureDescription
ReAct_Title_Feature_FeatureDescription_Reference = pd.merge(ReAct_Title_Feature_FeatureDescription, ReActAndReference, on='ReACT_number', how='outer')
ReAct_Title_Feature_FeatureDescription_Reference

,ReACT_number,ReACT_title,Features,FeatureDescription,References
0,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",num_act_dev,The number of Active Developers is the count o...,No. of reference: 3\n
1,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",top_e_fract,The percentages of emails performed by the top...,No. of reference: 3\n
2,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",top_c_fract,The percentages of Commits performed by the to...,No. of reference: 3\n
3,ReACT-1,Maintaining a set of active/friendly developers,num_act_dev,The number of Active Developers is the count o...,No. of reference: 2\n
4,ReACT-6,Maintaining Continuous Integration,num_act_dev,The number of Active Developers is the count o...,No. of reference: 2\n
...,...,...,...,...,...
75,ReACT-47,"Have the project owned by users, rather than\n...",top_e_fract,The percentages of emails performed by the top...,No. of reference: 1
76,ReACT-47,"Have the project owned by users, rather than\n...",top_c_fract,The percentages of Commits performed by the to...,No. of reference: 1
77,ReACT-48,Plan to finish the project in a shorter amount...,top_e_fract,The percentages of emails performed by the top...,No. of reference: 1
78,ReACT-48,Plan to finish the project in a shorter amount...,top_c_fract,The percentages of Commits performed by the to...,No. of reference: 1


In [62]:
df = ReAct_Title_Feature_FeatureDescription_Reference
ReACT_number = df['ReACT_number'].values
ReACT_Title = df['ReACT_title'].values
Feature = df['Features'].values
df['FeatureDescription'] = df['FeatureDescription'].apply(wrap_text)
FeatureDescription = df['FeatureDescription'].values

Edges = ReAct_Title_Feature_FeatureDescription_Reference['References'].values



net = Network(directed =True)
for i in range(0,len(ReACT_number)):
 # print(actionable[i])
 net.add_node(ReACT_number[i], label=ReACT_number[i], title=ReACT_Title[i])
 net.add_node(Feature[i], label=Feature[i], title=FeatureDescription[i], color = "#E07772")
 net.add_edge(ReACT_number[i], Feature[i], weight=extract_numbers(Edges[i]), title=Edges[i], color = "#898980")

#  net.add_edge(ReACT_number[i], Feature[i], weight=extract_numbers(Edges[i]), title=Edges[i], color = "#898980")


net.show('interactive_graph_Feature.html')